# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra data modeling

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Execute Function

In [6]:
def execute_query(session, *query):
    try:
        return session.execute(*query)
    except Exception as e:
        print(e)
    return None

#### Create Keyspace

In [7]:
# Create the keyspace sparkify
query = """
CREATE KEYSPACE IF NOT EXISTS sparkify 
WITH REPLICATION = 
{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
"""
execute_query(session, query)

#### Set Keyspace

In [8]:
# Set keyspace spakify specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession = 4

In [9]:
## Drop table
query = "DROP TABLE IF EXISTS music_by_session"
execute_query(session, query)

## Create table
query = """
CREATE TABLE IF NOT EXISTS music_by_session (
    session_id INT,
    session_item INT,
    artist_name TEXT,
    song_title TEXT,
    song_length FLOAT,
    PRIMARY KEY (session_id, session_item)
)
"""
execute_query(session, query)

In [10]:
# Process csv file and load music_by_session table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_by_session (session_id, session_item, artist_name, song_title, song_length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        execute_query(session, query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [11]:
# SELECT statement to verify the data was entered into the music_by_session table
query = """
SELECT
    artist_name,
    song_title,
    song_length
FROM
    music_by_session
WHERE
    session_id = 338 AND
    session_item = 4
"""
rows = execute_query(session, query)

for row in rows:
    print (row.artist_name, '|', row.song_title, '|', row.song_length)

Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
## Drop table
query = "DROP TABLE IF EXISTS music_user_by_user_session"
execute_query(session, query)

## Create table
query = """
CREATE TABLE IF NOT EXISTS music_user_by_user_session (
    user_id INT,
    session_id INT,
    session_item INT,
    artist_name TEXT,
    song_title TEXT,
    user_first_name TEXT,
    user_last_name TEXT,
    PRIMARY KEY ((user_id, session_id), session_item)
)
"""
execute_query(session, query)

In [13]:
# Process csv file and load music_user_by_user_session table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_user_by_user_session (user_id, session_id, session_item, artist_name, song_title, user_first_name, user_last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        execute_query(session, query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [14]:
# SELECT statement to verify the data was entered into the music_user_by_user_session table
query = """
SELECT
    artist_name,
    song_title,
    user_first_name,
    user_last_name
FROM
    music_user_by_user_session
WHERE
    user_id = 10 AND
    session_id = 182
"""
rows = execute_query(session, query)

for row in rows:
    print (row.artist_name, '|', row.song_title, '|', row.user_first_name, '|', row.user_last_name)

Down To The Bone | Keep On Keepin' On | Sylvie | Cruz
Three Drives | Greece 2000 | Sylvie | Cruz
Sebastien Tellier | Kilometer | Sylvie | Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie | Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [15]:
## Drop table
query = "DROP TABLE IF EXISTS user_by_song"
execute_query(session, query)

## Create table
query = """
CREATE TABLE IF NOT EXISTS user_by_song (
    user_id INT,
    song_title TEXT,
    user_first_name TEXT,
    user_last_name TEXT,
    PRIMARY KEY (song_title, user_id)
)
"""
execute_query(session, query)

In [16]:
# Process csv file and load user_by_song table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_by_song (user_id, song_title, user_first_name, user_last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        execute_query(session, query, (int(line[10]), line[9], line[1], line[4]))

In [17]:
# SELECT statement to verify the data was entered into the user_by_song table
query = """
SELECT
    user_first_name,
    user_last_name
FROM
    user_by_song
WHERE
    song_title = 'All Hands Against His Own'
"""
rows = execute_query(session, query)

for row in rows:
    print (row.user_first_name, '|', row.user_last_name)

Jacqueline | Lynch
Tegan | Levine
Sara | Johnson


### Drop the tables before closing out the sessions

In [18]:
# Drop the tables before closing out the sessions
query = "DROP TABLE IF EXISTS music_by_session"
execute_query(session, query)

query = "DROP TABLE IF EXISTS music_user_by_user_session"
execute_query(session, query)

query = "DROP TABLE IF EXISTS user_by_song"
execute_query(session, query)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()